In [3]:
import pandas as pd
import numpy as np
from sklearn import ensemble
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from xgboost import XGBClassifier
!pip install catboost
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from scipy.sparse import lil_matrix
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import copy
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
import matplotlib.pyplot as plt
from lightgbm import LGBMClassifier
from imblearn.over_sampling import SMOTE 
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.neighbors import KNeighborsClassifier
import seaborn as sns


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [6]:
# !wget http://nlp.stanford.edu/data/glove.6B.zip
# !cp -r --recursive "/content/glove.6B.zip"     "/content/drive/My Drive/DataScienceCompetitions/glove_embeddings"
!cp -r --recursive  "/content/drive/My Drive/DataScienceCompetitions/glove_embeddings"   "/content/glove.6B.zip"

In [7]:
!unzip /content/glove.6B.zip

Archive:  /content/glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [78]:
!cp -r --recursive "/content/drive/My Drive/DataScienceCompetitions/AV_Independence_day/train.csv"  '/content/train.csv'
!cp -r --recursive "/content/drive/My Drive/DataScienceCompetitions/AV_Independence_day/test.csv"  '/content/test.csv'
train = pd.read_csv('/content/train.csv')
test  = pd.read_csv("/content/test.csv")

In [79]:
train['combined_text'] = train['TITLE'] + train['ABSTRACT']
test['combined_text'] = test['TITLE'] + test['ABSTRACT']

In [80]:
train.head()

,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance,combined_text
0,1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0,Reconstructing Subject-Specific Effect Maps P...
1,2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0,Rotation Invariance Neural Network Rotation i...
2,3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0,Spherical polyharmonics and Poisson kernels fo...
3,4,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0,0,1,0,0,0,A finite element approximation for the stochas...
4,5,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0,Comparative study of Discrete Wavelet Transfor...


In [81]:
print(train.ID.nunique())

20972


In [82]:
print(train.TITLE.nunique())
train.TITLE.value_counts()

20972


SPIRou Input Catalog: Activity, Rotation and Magnetic Field of Cool Dwarfs                                                                   1
Learning a Unified Control Policy for Safe Falling                                                                                           1
Testing the Young Neutron Star Scenario with Persistent Radio Emission Associated with FRB 121102                                            1
Privacy Preserving Identification Using Sparse Approximation with Ambiguization                                                              1
Evaluating Quality of Chatbots and Intelligent Conversational Agents                                                                         1
                                                                                                                                            ..
Calibrated Fairness in Bandits                                                                                                               1

In [83]:
print(train["Computer Science"].nunique())
print(train.Physics.nunique())
print(train.Mathematics.nunique())
print(train.Statistics.nunique())
print(train["Quantitative Biology"].nunique())
print(train["Quantitative Finance"].nunique())

2
2
2
2
2
2


In [84]:
print(train["Computer Science"].value_counts())
print(train["Physics"].value_counts())
print(train["Mathematics"].value_counts())
print(train["Statistics"].value_counts())
print(train["Quantitative Biology"].value_counts())
print(train["Quantitative Finance"].value_counts())

0    12378
1     8594
Name: Computer Science, dtype: int64
0    14959
1     6013
Name: Physics, dtype: int64
0    15354
1     5618
Name: Mathematics, dtype: int64
0    15766
1     5206
Name: Statistics, dtype: int64
0    20385
1      587
Name: Quantitative Biology, dtype: int64
0    20723
1      249
Name: Quantitative Finance, dtype: int64


In [86]:
df_train = pd.read_csv('/content/train.csv')
df_train['train_flag'] = 1
df_train['combined_text'] = df_train['TITLE'] + df_train['ABSTRACT']
print(df_train.shape)


df_test = pd.read_csv('/content/test.csv')
df_test['combined_text'] = df_test['TITLE'] + df_test['ABSTRACT']
df_test['train_flag'] = 0
df_test['Computer Science'] =    -1
df_test['Physics'] =             -1
df_test['Mathematics'] =         -1
df_test['Statistics'] =          -1
df_test['Quantitative Biology'] =-1
df_test['Quantitative Finance'] =-1

print(df_test.shape)

df_data = pd.concat([df_train,df_test])
print(df_data.shape)
df_data = df_data.reset_index(drop=True)
df_data.head()

(20972, 11)
(8989, 11)
(29961, 11)


,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance,train_flag,combined_text
0,1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0,1,Reconstructing Subject-Specific Effect Maps P...
1,2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0,1,Rotation Invariance Neural Network Rotation i...
2,3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0,1,Spherical polyharmonics and Poisson kernels fo...
3,4,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0,0,1,0,0,0,1,A finite element approximation for the stochas...
4,5,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0,1,Comparative study of Discrete Wavelet Transfor...


In [87]:
line = df_data["combined_text"].loc[[2]].values[0]
print(line)
print(str(line).lower().split())

Spherical polyharmonics and Poisson kernels for polyharmonic functions  We introduce and develop the notion of spherical polyharmonics, which are a
natural generalisation of spherical harmonics. In particular we study the
theory of zonal polyharmonics, which allows us, analogously to zonal harmonics,
to construct Poisson kernels for polyharmonic functions on the union of rotated
balls. We find the representation of Poisson kernels and zonal polyharmonics in
terms of the Gegenbauer polynomials. We show the connection between the
classical Poisson kernel for harmonic functions on the ball, Poisson kernels
for polyharmonic functions on the union of rotated balls, and the Cauchy-Hua
kernel for holomorphic functions on the Lie ball.

['spherical', 'polyharmonics', 'and', 'poisson', 'kernels', 'for', 'polyharmonic', 'functions', 'we', 'introduce', 'and', 'develop', 'the', 'notion', 'of', 'spherical', 'polyharmonics,', 'which', 'are', 'a', 'natural', 'generalisation', 'of', 'spherical', 'harm

In [89]:
line = df_data["combined_text"].loc[[5]].values[0]
print(line)
print(str(line).lower().split())

On maximizing the fundamental frequency of the complement of an obstacle  Let $\Omega \subset \mathbb{R}^n$ be a bounded domain satisfying a
Hayman-type asymmetry condition, and let $ D $ be an arbitrary bounded domain
referred to as "obstacle". We are interested in the behaviour of the first
Dirichlet eigenvalue $ \lambda_1(\Omega \setminus (x+D)) $. First, we prove an
upper bound on $ \lambda_1(\Omega \setminus (x+D)) $ in terms of the distance
of the set $ x+D $ to the set of maximum points $ x_0 $ of the first Dirichlet
ground state $ \phi_{\lambda_1} > 0 $ of $ \Omega $. In short, a direct
corollary is that if \begin{equation} \mu_\Omega := \max_{x}\lambda_1(\Omega
\setminus (x+D)) \end{equation} is large enough in terms of $ \lambda_1(\Omega)
$, then all maximizer sets $ x+D $ of $ \mu_\Omega $ are close to each maximum
point $ x_0 $ of $ \phi_{\lambda_1} $.
Second, we discuss the distribution of $ \phi_{\lambda_1(\Omega)} $ and the
possibility to inscribe wavelength balls at a g

In [90]:
line = df_data["combined_text"].loc[[5]].values[0]
print(line)
line = line.replace('"', '')
line = line.replace('\'', ' ')
line = line.replace(".",' ')
line = line.replace("#",' ')
line = line.replace("&",' and ')
line = line.replace("!",' ')
line = line.replace(";",' ')
line = line.replace(":",' ')
line = line.replace(",",' ')
line = line.replace("~",' ')
line = line.replace(")",' ')
line = line.replace("(",' ')
line = line.replace("}",' ')
line = line.replace("{",' ')
line = line.replace("[",' ')
line = line.replace("]",' ')

print(line)
sentence = str(line).lower().split()
print(sentence)

for ind,word in enumerate(sentence):
    if '\\' in word or "/" in word  or "$" in word  or "_" in word or word[0]=="-" or "*" in word or "+" in word or "@" in word  :
        sentence[ind] = "CONST"
print(sentence)

while 1:
    if "CONST" in sentence:
        sentence.remove("CONST")
    else:
        break
print(sentence)

On maximizing the fundamental frequency of the complement of an obstacle  Let $\Omega \subset \mathbb{R}^n$ be a bounded domain satisfying a
Hayman-type asymmetry condition, and let $ D $ be an arbitrary bounded domain
referred to as "obstacle". We are interested in the behaviour of the first
Dirichlet eigenvalue $ \lambda_1(\Omega \setminus (x+D)) $. First, we prove an
upper bound on $ \lambda_1(\Omega \setminus (x+D)) $ in terms of the distance
of the set $ x+D $ to the set of maximum points $ x_0 $ of the first Dirichlet
ground state $ \phi_{\lambda_1} > 0 $ of $ \Omega $. In short, a direct
corollary is that if \begin{equation} \mu_\Omega := \max_{x}\lambda_1(\Omega
\setminus (x+D)) \end{equation} is large enough in terms of $ \lambda_1(\Omega)
$, then all maximizer sets $ x+D $ of $ \mu_\Omega $ are close to each maximum
point $ x_0 $ of $ \phi_{\lambda_1} $.
Second, we discuss the distribution of $ \phi_{\lambda_1(\Omega)} $ and the
possibility to inscribe wavelength balls at a g

In [91]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29961 entries, 0 to 29960
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   ID                    29961 non-null  int64 
 1   TITLE                 29961 non-null  object
 2   ABSTRACT              29961 non-null  object
 3   Computer Science      29961 non-null  int64 
 4   Physics               29961 non-null  int64 
 5   Mathematics           29961 non-null  int64 
 6   Statistics            29961 non-null  int64 
 7   Quantitative Biology  29961 non-null  int64 
 8   Quantitative Finance  29961 non-null  int64 
 9   train_flag            29961 non-null  int64 
 10  combined_text         29961 non-null  object
dtypes: int64(8), object(3)
memory usage: 2.5+ MB


In [92]:
from nltk.tokenize import RegexpTokenizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [93]:
# here u will get an array with various strings seperated by space
tkr = RegexpTokenizer('[a-zA-Z]+')
abs_list = []
computer = []
physics = []
maths = []
stats = []
bio = []
finance = []

abs_test = []
colID = []


for i in range(len(df_data)):
    line = df_data["combined_text"][i]

    line = line.replace('"', '')
    line = line.replace('\'', ' ')
    line = line.replace(".",' ')
    line = line.replace("#",' ')
    line = line.replace("&",' and ')
    line = line.replace("!",' ')
    line = line.replace(";",' ')
    line = line.replace(":",' ')
    line = line.replace(",",' ')
    line = line.replace("~",' ')
    line = line.replace(")",' ')
    line = line.replace("(",' ')
    line = line.replace("}",' ')
    line = line.replace("{",' ')
    line = line.replace("[",' ')
    line = line.replace("]",' ')

    sentence = str(line).lower().split()

    for ind,word in enumerate(sentence):
        if '\\' in word or "/" in word  or "$" in word  or "_" in word or word[0]=="-" or "*" in word or "+" in word or "@" in word  :
            sentence[ind] = "CONST"

    while 1:
        if "CONST" in sentence:
            sentence.remove("CONST")
        else:
            break



    sentence = tkr.tokenize(str(sentence))




    if df_data["train_flag"][i] == 1:
        abs_list.append(sentence)

        computer.append(df_data["Computer Science"][i])
        physics.append(df_data["Physics"][i])
        maths.append(df_data["Mathematics"][i])
        stats.append(df_data["Statistics"][i])
        bio.append(df_data["Quantitative Biology"][i])
        finance.append(df_data["Quantitative Finance"][i])
    else:
        abs_test.append(sentence)
        colID.append(df_data["ID"][i])
        


print(abs_list[3])
print(len(abs_list))
print(abs_test[3])
print(len(abs_test))

print(computer[:10])
print(physics[:10])
print(maths[:10])
print(stats[:10])
print(bio[:10])
print(finance[:10])

['a', 'finite', 'element', 'approximation', 'for', 'the', 'stochastic', 'maxwell', 'landau', 'lifshitz', 'gilbert', 'system', 'the', 'stochastic', 'landau', 'lifshitz', 'gilbert', 'llg', 'equation', 'coupled', 'with', 'the', 'maxwell', 'equations', 'the', 'so', 'called', 'stochastic', 'mllg', 'system', 'describes', 'the', 'creation', 'of', 'domain', 'walls', 'and', 'vortices', 'fundamental', 'objects', 'for', 'the', 'novel', 'nanostructured', 'magnetic', 'memories', 'we', 'first', 'reformulate', 'the', 'stochastic', 'llg', 'equation', 'into', 'an', 'equation', 'with', 'time', 'differentiable', 'solutions', 'we', 'then', 'propose', 'a', 'convergent', 'scheme', 'to', 'approximate', 'the', 'solutions', 'of', 'the', 'reformulated', 'system', 'as', 'a', 'consequence', 'we', 'prove', 'convergence', 'of', 'the', 'approximate', 'solutions', 'with', 'no', 'or', 'minor', 'conditions', 'on', 'time', 'and', 'space', 'steps', 'depending', 'on', 'the', 'value', 'of', 'hence', 'we', 'prove', 'the', '

In [25]:
# aa = df_data["ABSTRACT"][0]
# print(aa)
# print(len(aa.split()) , aa.split())
# print(len(aa.lower().split())  ,aa.lower().split())
# tkr = RegexpTokenizer('[a-zA-Z]+')
# zz = str(aa.lower().split())
# print(len(zz) , zz)
# print(len(tkr.tokenize(zz)) , tkr.tokenize(zz))
# tkr.tokenize("[dg fb][n]")

In [94]:
# here, u will get a sequence for each word in all strings
tokenizer = Tokenizer()

tokenizer.fit_on_texts(abs_list)
sequenceTrain = tokenizer.texts_to_sequences(abs_list)
print(sequenceTrain[0])
print(len(sequenceTrain))



tokenizer.fit_on_texts(abs_test)
sequenceTest = tokenizer.texts_to_sequences(abs_test)
print(sequenceTest[0])
print(len(sequenceTest))


[4451, 1155, 326, 214, 700, 1029, 42, 842, 1155, 326, 284, 69, 1362, 1537, 294, 8309, 6, 4860, 22, 121, 4, 1155, 49, 22, 284, 21, 19, 805, 26, 34, 866, 342, 112, 108, 33122, 518, 495, 8, 1, 1155, 3, 151, 112, 108, 1283, 518, 214, 12, 103, 520, 727, 1720, 18, 1, 1155, 49, 22, 116, 342, 284, 9, 786, 62, 151, 284, 20, 21, 19, 62, 5, 216, 1155, 326, 214, 700, 9, 3743, 62, 554, 194, 42, 321, 1433, 1090, 4374, 2108, 2, 7883, 2207, 6455, 6, 13, 675, 7, 81, 4, 795, 38, 2111, 12663, 5, 397, 1155, 326, 4374, 2, 1029, 329, 209, 3, 6, 171, 561, 1281, 12663, 678, 1376, 5, 717, 245, 181, 5, 352, 224, 277, 11, 27, 4, 146, 221, 2, 289, 5, 943, 1281, 1, 60, 38, 9, 4, 13439, 144, 53, 10, 21, 19, 62, 11, 65, 561, 1281, 6, 4, 3509, 1357, 112, 108, 218, 79, 12, 518, 495, 795, 9, 2829, 16, 4, 433, 4, 5066, 40, 11, 4, 515, 23, 545, 141, 15, 1030, 18, 129, 22, 6, 4, 958, 326, 2251, 247, 511, 9, 690, 12, 8519, 391, 22, 3, 22, 18, 1, 5067, 49, 1537, 4860, 7416, 11059, 1452, 33, 12, 792, 22, 122, 10, 27, 12663, 

In [95]:
from collections import defaultdict
ma = 0
st = ""
count = defaultdict(int)
for i in sequenceTrain:
    count[len(i)] = count[len(i)] + 1
    if ma < len(i):
        ma = len(i)
        st = i

print(ma)
print(st)
print(count)

# ma = 0
# st = ""
# count = defaultdict(int)
# for i in sequenceTest:
#     count[len(i)] = count[len(i)] + 1
#     if ma < len(i):
#         ma = len(i)
#         st = i

# print(ma)
# print(st)
# print(count)

458
[17, 2209, 5, 392, 22, 57, 498, 3, 669, 1164, 8, 22, 3550, 392, 22, 57, 11489, 9, 4, 203, 3, 446, 1661, 25926, 920, 4, 90, 2, 51, 392, 3, 721, 777, 5, 1960, 748, 158, 8, 1682, 157, 22, 13, 35, 9, 4, 3598, 2209, 128, 4, 564, 1310, 1912, 5, 982, 498, 3, 669, 1164, 2, 11489, 8, 63, 2421, 2209, 3, 3132, 392, 22, 57, 11489, 9, 4, 203, 25926, 10, 3409, 18, 238, 903, 6, 286, 872, 1000, 3, 270, 605, 336, 1, 9791, 2977, 2, 1, 4228, 798, 50, 21, 1840, 1627, 721, 209, 8, 22, 57, 2017, 859, 6, 1, 1178, 11489, 6606, 4872, 16, 4, 25926, 11, 1, 8729, 903, 2, 27291, 625, 570, 3, 48978, 3, 18770, 6, 2469, 1959, 3, 168, 19503, 6, 4, 6256, 35, 6, 18770, 11489, 9, 1725, 1132, 14, 1, 1222, 10, 1000, 3, 605, 113, 4, 1243, 46, 5, 1960, 308, 2083, 3, 2987, 1071, 79, 220, 1, 91, 2, 22, 2331, 108, 143, 48979, 11489, 1376, 26, 920, 71, 9811, 962, 262, 3, 1806, 56, 5, 1960, 505, 3, 1532, 1, 157, 392, 3, 721, 291, 509, 22, 10, 15, 321, 1539, 16, 154, 1842, 6, 1541, 36, 70, 137, 440, 377, 336, 1, 1334, 564, 730

In [97]:
for i in range(458):
    print(count[457-i])
    if i%100==0:
        print()
        print()
# 140 must be reduce


0


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0


0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
1
1
0
0
0
0
2
2
3
4
1
1
5
7
1
4
5
2
2
6
2
5
4
4
12
9
9
9
11
14
9
8
21
12
22
11
22
18
17
15
18
23
18
25
16
22
18
27
29
24
30
31
24
31
24
31
28
27
32
30
33
31
25
37
39
35
42
40
47
34
36
35
42
48
45
48
38
39
41
47
46


55
47
43
49
55
30
54
57
42
45
49
62
50
48
50
46
38
58
63
62
59
50
67
60
62
59
65
62
66
74
62
72
59
50
75
83
60
79
93
74
96
89
85
91
102
89
100
87
89
78
109
100
111
78
88
89
92
101
95
98
92
100
84
100
100
101
120
123
87
98
102
96
108
108
101
104
107
101
110
125
124
133
119
111
131
130
125
129
118
125
125
143
142
108
141
136
135
140
119
150


147
130
138
161
141
139
131
127
131
115
144
139
118
138
135
136
137
119
112
112
125
124
118
116
131
122
119
116
111
106
111
113
125
143
119
124
128
132
131
119
121
89
117
122
118
105
106
108
114
10

In [100]:
#add padding from the starting, so list could be equal in size to 40
maxlentweet = 350
sequenceTrain = pad_sequences(sequenceTrain, maxlen=maxlentweet)
print(sequenceTrain[40])
print(sequenceTrain.shape)
print(len(sequenceTrain))

sequenceTest = pad_sequences(sequenceTest, maxlen=maxlentweet)
print(sequenceTest[0])
print(sequenceTest.shape)
print(len(sequenceTest))

[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0  6174   895     3   653  2008   

In [101]:
embeddings_index = {}
with open("/content/glove.6B.100d.txt") as f:
    for line in f.readlines():
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print('Total number of word vectors are:' , len(embeddings_index))





max_words = len(embeddings_index)
embedding_dim = 100
embedding_matrix = np.zeros((max_words, embedding_dim))
count = 0
for word, i in embeddings_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[count] = embedding_vector
        count+=1

embedding_matrix.shape

Total number of word vectors are: 400000


(400000, 100)

In [102]:
from keras import models
from keras import layers
from keras import optimizers
import tensorflow as tf

from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation,SpatialDropout1D,GRU

from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D,Conv1D,GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers

In [103]:
sequenceTrain = np.array(sequenceTrain)
computer = np.array(computer)
maths = np.array(maths)
physics = np.array(physics)
stats = np.array(stats)
bio = np.array(bio)
finance = np.array(finance)

# train_size = int(len(sequenceTrain) * 0.8)
# train_x = sequenceTrain[:train_size]
# train_y = computer[:train_size]
# val_x = sequenceTrain[train_size:]
# val_y = computer[train_size:]
# print(train_x.shape , train_y.shape)
# print(val_x.shape , val_y.shape)

In [104]:
import keras.backend as K
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [105]:
# max_words = 400000
# embedding_dim = 100

train_X = sequenceTrain
train_Y = computer
print(train_X.shape , train_Y.shape)


modelC = tf.keras.Sequential([Input(shape=(maxlentweet,)),
                             Embedding(max_words, embedding_dim, weights=[embedding_matrix]),
                             Bidirectional(LSTM(50, return_sequences=True, dropout=0.2)),
                             Bidirectional(LSTM(50, return_sequences=True, dropout=0.2)),
                             GlobalMaxPool1D(),
                             Dense(50, activation="relu"),
                             Dropout(0.2),
                             Dense(1, activation="sigmoid")
                            ])

modelC.layers[0].trainable = False
print(modelC.summary())

modelC.compile(loss='binary_crossentropy' , optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3) ,metrics=['accuracy',get_f1])
historyC = modelC.fit(train_X,
                  train_Y,
                  epochs = 3,
                #   batch_size=2048,
                  validation_split=0.1
                #   validation_data=(val_x,val_y),
                #   callbacks=[earlystopper,reduce_lr]
                  )

(20972, 350) (20972,)
Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 350, 100)          40000000  
_________________________________________________________________
bidirectional_18 (Bidirectio (None, 350, 100)          60400     
_________________________________________________________________
bidirectional_19 (Bidirectio (None, 350, 100)          60400     
_________________________________________________________________
global_max_pooling1d_9 (Glob (None, 100)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 50)                5050      
_________________________________________________________________
dropout_12 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_22 (Dense)             (N

In [106]:
# max_words = 400000
# embedding_dim = 100

train_X = sequenceTrain
train_Y = physics
print(train_X.shape , train_Y.shape)


modelP = tf.keras.Sequential([Input(shape=(maxlentweet,)),
                             Embedding(max_words, embedding_dim, weights=[embedding_matrix]),
                             Bidirectional(LSTM(50, return_sequences=True, dropout=0.2)),
                             Bidirectional(LSTM(50, return_sequences=True, dropout=0.2)),
                             GlobalMaxPool1D(),
                             Dense(50, activation="relu"),
                             Dropout(0.2),
                             Dense(1, activation="sigmoid")
                            ])


modelP.layers[0].trainable = False
print(modelP.summary())

modelP.compile(loss='binary_crossentropy' , optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3) , metrics=['accuracy',get_f1])

historyP = modelP.fit(train_X,
                  train_Y,
                  epochs = 12,
                  validation_split=0.1
                #   batch_size=2048,
                #   validation_data=(val_x,val_y),
                #   callbacks=[earlystopper,reduce_lr]
                  )

(20972, 350) (20972,)
Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 350, 100)          40000000  
_________________________________________________________________
bidirectional_20 (Bidirectio (None, 350, 100)          60400     
_________________________________________________________________
bidirectional_21 (Bidirectio (None, 350, 100)          60400     
_________________________________________________________________
global_max_pooling1d_10 (Glo (None, 100)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 50)                5050      
_________________________________________________________________
dropout_13 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_24 (Dense)             (N

In [107]:
# max_words = 400000
# embedding_dim = 100

train_X = sequenceTrain
train_Y = maths
print(train_X.shape , train_Y.shape)


modelM = tf.keras.Sequential([Input(shape=(maxlentweet,)),
                             Embedding(max_words, embedding_dim, weights=[embedding_matrix]),
                             Bidirectional(LSTM(50, return_sequences=True, dropout=0.2)),
                             Bidirectional(LSTM(50, return_sequences=True, dropout=0.2)),
                             GlobalMaxPool1D(),
                             Dense(50, activation="relu"),
                             Dropout(0.2),
                             Dense(1, activation="sigmoid")
                            ])

modelM.layers[0].trainable = False
print(modelM.summary())

modelM.compile(loss='binary_crossentropy' , optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3) , metrics=['accuracy',get_f1])

historyM = modelM.fit(train_X,
                  train_Y,
                  epochs = 10,
                  validation_split=0.1
                #   batch_size=2048,
                #   validation_data=(val_x,val_y),
                #   callbacks=[earlystopper,reduce_lr]
                  )

(20972, 350) (20972,)
Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 350, 100)          40000000  
_________________________________________________________________
bidirectional_22 (Bidirectio (None, 350, 100)          60400     
_________________________________________________________________
bidirectional_23 (Bidirectio (None, 350, 100)          60400     
_________________________________________________________________
global_max_pooling1d_11 (Glo (None, 100)               0         
_________________________________________________________________
dense_25 (Dense)             (None, 50)                5050      
_________________________________________________________________
dropout_14 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_26 (Dense)             (N

In [108]:
# max_words = 400000
# embedding_dim = 100

train_X = sequenceTrain
train_Y = stats
print(train_X.shape , train_Y.shape)

modelS = tf.keras.Sequential([Input(shape=(maxlentweet,)),
                             Embedding(max_words, embedding_dim, weights=[embedding_matrix]),
                             Bidirectional(LSTM(50, return_sequences=True, dropout=0.2)),
                             Bidirectional(LSTM(50, return_sequences=True, dropout=0.2)),
                             GlobalMaxPool1D(),
                             Dense(50, activation="relu"),
                             Dropout(0.2),
                             Dense(1, activation="sigmoid")
                            ])

modelS.layers[0].trainable = False
print(modelS.summary())

modelS.compile(loss='binary_crossentropy' , optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3)  , metrics=['accuracy',get_f1])

historyS = modelS.fit(train_X,
                  train_Y,
                  epochs = 10,
                  validation_split=0.1
                #   batch_size=2048,
                #   validation_data=(val_x,val_y),
                #   callbacks=[earlystopper,reduce_lr]
                  )

(20972, 350) (20972,)
Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, 350, 100)          40000000  
_________________________________________________________________
bidirectional_24 (Bidirectio (None, 350, 100)          60400     
_________________________________________________________________
bidirectional_25 (Bidirectio (None, 350, 100)          60400     
_________________________________________________________________
global_max_pooling1d_12 (Glo (None, 100)               0         
_________________________________________________________________
dense_27 (Dense)             (None, 50)                5050      
_________________________________________________________________
dropout_15 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_28 (Dense)             (N

In [109]:
# max_words = 400000
# embedding_dim = 100

train_X = sequenceTrain
train_Y = bio
print(train_X.shape , train_Y.shape)

modelB = tf.keras.Sequential([Input(shape=(maxlentweet,)),
                             Embedding(max_words, embedding_dim, weights=[embedding_matrix]),
                             Bidirectional(LSTM(50, return_sequences=True, dropout=0.2)),
                             Bidirectional(LSTM(50, return_sequences=True, dropout=0.2)),
                             GlobalMaxPool1D(),
                             Dense(50, activation="relu"),
                             Dropout(0.2),
                             Dense(1, activation="sigmoid")
                            ])

modelB.layers[0].trainable = False
print(modelB.summary())

modelB.compile(loss='binary_crossentropy' , optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3)  , metrics=['accuracy',get_f1])

historyB = modelB.fit(train_X,
                  train_Y,
                  epochs = 5,
                  validation_split=0.1
                #   batch_size=2048,
                #   validation_data=(val_x,val_y),
                #   callbacks=[earlystopper,reduce_lr]
                  )

(20972, 350) (20972,)
Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_16 (Embedding)     (None, 350, 100)          40000000  
_________________________________________________________________
bidirectional_26 (Bidirectio (None, 350, 100)          60400     
_________________________________________________________________
bidirectional_27 (Bidirectio (None, 350, 100)          60400     
_________________________________________________________________
global_max_pooling1d_13 (Glo (None, 100)               0         
_________________________________________________________________
dense_29 (Dense)             (None, 50)                5050      
_________________________________________________________________
dropout_16 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_30 (Dense)             (N

In [110]:
# max_words = 400000
# embedding_dim = 100

train_X = sequenceTrain
train_Y = finance
print(train_X.shape , train_Y.shape)

modelF = tf.keras.Sequential([Input(shape=(maxlentweet,)),
                             Embedding(max_words, embedding_dim, weights=[embedding_matrix]),
                             Bidirectional(LSTM(50, return_sequences=True, dropout=0.2)),
                             Bidirectional(LSTM(50, return_sequences=True, dropout=0.2)),
                             GlobalMaxPool1D(),
                             Dense(50, activation="relu"),
                             Dropout(0.2),
                             Dense(1, activation="sigmoid")
                            ])

modelF.layers[0].trainable = False
print(modelF.summary())

modelF.compile(loss='binary_crossentropy' , optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3)  , metrics=['accuracy',get_f1])

historyF = modelF.fit(train_X,
                  train_Y,
                  epochs = 5,
                  validation_split=0.1
                #   batch_size=2048,
                #   validation_data=(val_x,val_y),
                #   callbacks=[earlystopper,reduce_lr]
                  )

(20972, 350) (20972,)
Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_17 (Embedding)     (None, 350, 100)          40000000  
_________________________________________________________________
bidirectional_28 (Bidirectio (None, 350, 100)          60400     
_________________________________________________________________
bidirectional_29 (Bidirectio (None, 350, 100)          60400     
_________________________________________________________________
global_max_pooling1d_14 (Glo (None, 100)               0         
_________________________________________________________________
dense_31 (Dense)             (None, 50)                5050      
_________________________________________________________________
dropout_17 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_32 (Dense)             (N

In [113]:
C = []
val = 0.4
for ss in modelC.predict(sequenceTest):
    if ss>val:
        C.append(1)
    else:
        C.append(0)


P = []
for ss in modelP.predict(sequenceTest):
    if ss>val:
        P.append(1)
    else:
        P.append(0)


M = []
for ss in modelM.predict(sequenceTest):
    if ss>val:
        M.append(1)
    else:
        M.append(0)


S = []
for ss in modelS.predict(sequenceTest):
    if ss>val:
        S.append(1)
    else:
        S.append(0)


B = []
for ss in modelB.predict(sequenceTest):
    if ss>val:
        B.append(1)
    else:
        B.append(0)

F = []
for ss in modelF.predict(sequenceTest):
    if ss>val:
        F.append(1)
    else:
        F.append(0)

In [114]:
dic = {'ID': colID, 'Computer Science': C , 'Physics':P, 'Mathematics':M, 'Statistics':S, 'Quantitative Biology':B, 'Quantitative Finance':F} 
df_fin = pd.DataFrame(dic) 
filename = "sub.csv"
df_fin.to_csv(filename , index=False)
from google.colab import files
files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# tt = pd.read_csv("/content/test.csv")
# col1 = []
# for i in range(len(sequenceTest)):
#     col1.append(tt["ID"][i])

20973

In [ ]:
print(sequenceTest.shape)
print(sequenceTest[0].shape)

(8989, 300)
(300,)


In [ ]:
ind = 2
print(modelC.predict(sequenceTest[ind].reshape(1,300)))
print(modelP.predict(sequenceTest[ind].reshape(1,300)))
print(modelM.predict(sequenceTest[ind].reshape(1,300)))
print(modelS.predict(sequenceTest[ind].reshape(1,300)))
print(modelB.predict(sequenceTest[ind].reshape(1,300)))
print(modelF.predict(sequenceTest[ind].reshape(1,300)))

[[0.3899062]]
[[0.56794864]]
[[0.51846033]]
[[0.21701778]]
[[0.596456]]
[[0.6026816]]


In [ ]:
def train_model(X , Y):
    # classifier = LGBMClassifier()
    # param_grid = {"max_depth":[4,6,8,10,-1],
    #       "num_leaves":[7,15,31,63,127],
    #       "min_data_in_leaf":[1,3,5,7,10,25],
    #       "n_estimators":list(range(50,1001,100))}

    # model = RandomizedSearchCV(estimator = classifier,
    #                     param_distributions = param_grid,
    #                     cv = 5,
    #                     n_iter = 10,
    #                     verbose=1000
    #                     )

    # classifier = XGBClassifier(n_jobs = -1)
    # param_grid = {
    #     "n_estimators" : np.arange(100,1200,100),
    #     "max_depth" : np.arange(3,10)
    # }

    # model = RandomizedSearchCV(estimator = classifier,
    #                     param_distributions = param_grid,
    #                     cv = 5,
    #                     n_iter = 10,
    #                     verbose=1000
    #                     )

    # classifier = CatBoostClassifier(verbose=0)
    # param_grid = {
    #     "n_estimators" : np.arange(100,1200,100),
    #     "depth" : np.arange(3,10)
    # }

    # model = RandomizedSearchCV(estimator = classifier,
    #                     param_distributions = param_grid,
    #                     cv = 5,
    #                     n_iter = 10,
    #                     verbose=1000
    #                     )

    model = CatBoostClassifier(n_estimators=1100, depth=3, verbose=0)
    # model = RandomForestClassifier()
    # model = XGBClassifier(n_estimators=800 , max_depth=5 , verbose=10)
    # model = LGBMClassifier(max_depth=30 , n_estimators=500 , num_leaves=31)
    model.fit(X , Y)
    return model



def prediction(model):
    drop = copy.deepcopy(col_to_drop)
    X = test.drop(drop , axis=1)
    print(X.shape)

    col = model.predict_proba(X)
    # col = model.predict_log_proba(X)

    return col





In [ ]:
# Offered_By	Category	Rating	Reviews	Size	Price	Content_Rating	Last_Updated_On	Release_Version	OS_Version_Required	Downloads	train_flag
col_to_drop = ["Offered_By", "Last_Updated_On", "train_flag", "Downloads"]



X = train.drop(col_to_drop , axis=1)
Y = train["Downloads"]
print(X.shape, Y.shape)
m1 = train_model(X , Y)



(16516, 8) (16516,)


In [ ]:
print(m1.score(X,Y))
col = prediction(m1)
print(col.shape)




# col4 = []
# if col3.shape == (len(df_test),1):
#     for i in range(len(df_test)):
#         col4.append(col3[i][0])
#     col3 = copy.deepcopy(col4)
# print(np.array(col3).shape)




0.6078348268345847
(24776, 8)
(24776, 18)


In [ ]:
a1 = pd.read_csv("/content/lgbt1.csv")
a2 = pd.read_csv("/content/lgbt2.csv")
b1 = pd.read_csv("/content/xg1.csv")
b2 = pd.read_csv("/content/xg2.csv")
c1 = pd.read_csv("/content/ran1.csv")
c2 = pd.read_csv("/content/ran2.csv")
d1 = pd.read_csv("/content/cat1.csv")
d2 = pd.read_csv("/content/cat2.csv")

In [ ]:
a1["breed_category"][0]

1.0

In [ ]:
col1 , col2 , col3 = [] , [] , []
for i in range(len(a1)):
    arg1 = a1["breed_category"][i]
    arg2 = a2["breed_category"][i]
    arg3 = b1["breed_category"][i]
    arg4 = b2["breed_category"][i]
    arg5 = c1["breed_category"][i]
    arg6 = c2["breed_category"][i]
    arg7 = d1["breed_category"][i]
    arg8 = d2["breed_category"][i]
    # lis1 = [arg1, arg2, arg3, arg4, arg5, arg6, arg7, arg8]
    lis1 = [ arg2, arg4 , arg8]
    ans1 = max(set(lis1), key = lis1.count) 





    col1.append(a1["pet_id"][i])
    col2.append(ans1)
    col3.append(ans2)


dict = {'pet_id': col1, 'breed_category': col2 , 'pet_category':col3} 
df_fin = pd.DataFrame(dict) 
filename = "qqqqqqqqqqqqq.csv"
df_fin.to_csv(filename , index=False)
from google.colab import files
files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>